In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import hsv
import matplotlib.patches as mpatches
!pip install pmdarima
from pmdarima.arima import auto_arima, ADFTest

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
processed_df = pd.read_csv('/kaggle/input/stock-exchange-data/indexProcessed.csv')
data_df = pd.read_csv('/kaggle/input/stock-exchange-data/indexProcessed.csv')
info_df = pd.read_csv('/kaggle/input/stock-exchange-data/indexInfo.csv')

In [ ]:
processed_df.describe()

In [ ]:
processed_df.describe()

In [ ]:
processed_df = processed_df.merge(info_df, on='Index')
processed_df

In [ ]:
processed_df.isnull().any()

In [ ]:
print(processed_df.dtypes)

In [ ]:
processed_df['Date'] = pd.to_datetime(processed_df['Date'])
processed_df['Index'] = processed_df['Index'].astype('category')
processed_df['Region'] = processed_df['Region'].astype('category')
processed_df['Exchange'] = processed_df['Exchange'].astype('category')
processed_df['Currency'] = processed_df['Currency'].astype('category')

<font size="5">Let's make a plot of the closing prices of the stocks across time</font>

In [ ]:
stocks = pd.unique(processed_df['Index'])
stock_dfs = []
for stock in stocks:
    stock_dfs.append(processed_df[processed_df['Index'] == stock])
stock_dfs[0]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))
patches = []
for i, stock_df in enumerate(stock_dfs):
    color = (hsv(i/len(stock_dfs)))
    sns.lineplot(ax=ax, x=stock_df['Date'], y=stock_df['CloseUSD'], color=color)
    patches.append(mpatches.Patch(color=color, label=stock_df['Index'].iloc[0]))
ax.legend(handles=patches)

stock_dfs = { stock_df['Index'].iloc[0]: stock_df for stock_df in stock_dfs } # Convert list to dict
# sns.lineplot(ax=ax, data=processed_df, x='Date', y='CloseUSD')
# fig.show()

Stationarity Test

In [ ]:
adf_test = ADFTest(alpha=0.05)
for stock in stocks:
    print(stock,adf_test.should_diff(stock_dfs[stock][['CloseUSD', 'Date']].set_index('Date')))

In [ ]:
stock_dfs['HSI'][['CloseUSD', 'Date']].set_index('Date').plot()

In [ ]:
NSEI = stock_dfs['NSEI']
NSEI = NSEI.sort_values('Date').reset_index(drop=True)
train_df = NSEI[NSEI['Date'] < '2018'][['CloseUSD', 'Date']].set_index('Date')
test_df = NSEI[NSEI['Date'] > '2018'][['CloseUSD', 'Date']].set_index('Date')
plt.plot(train_df)
plt.plot(test_df)
plt.legend(['train', 'test'])

In [ ]:
arima_model = auto_arima(train_df)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods=len(test_df)), index=test_df.index)
prediction.columns = ['CloseUSD']
prediction

In [ ]:
plt.figure(figsize=(10,10))
sns.lineplot(x=train_df.index, y=train_df['CloseUSD'])
sns.lineplot(x=test_df.index, y=test_df['CloseUSD'])
sns.lineplot(x=prediction.index, y=prediction['CloseUSD'])
plt.legend(['train', 'test', 'prediction'])